In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from time import strptime
import datetime
from datetime import datetime
import warnings
import time
import csv
import glob
import logging
import importlib
import json
import networkx as nx
import seaborn as sns
import re
from operator import itemgetter
import matplotlib.cm as mcm
import community
from community import community_louvain as cm
import leidenalg as la
import igraph as ig
from cdlib import algorithms

In [3]:
path = glob.glob("F:/Supsi/COVID-19-Tweets/large_files/rt_network/leiden/network_12/")

In [4]:
path_nt = glob.glob("F:/Supsi/COVID-19-Tweets/large_files/rt_network/names/")
df = pd.read_csv(path_nt[0] + "user_rtuser.csv", lineterminator="\n", low_memory=False)
df

,name,rt_name
0,Huerconetzin,AnneKPIX
1,JustAnotherAme4,cnni
2,HHSRegion8,CDCgov
3,Paxman42,OurWarOnCancer
4,beerhowell,nytimes
...,...,...
48312499,Dinora21971394,saludhn
48312500,EllyJonkerVliet,StopEcocideNL
48312501,anna76746180,Haggis_UK
48312502,SueHealy16,Auchindrain


In [32]:
retweets_graph = nx.from_pandas_edgelist(df,'name','rt_name', create_using=nx.DiGraph())
print(nx.info(retweets_graph)) #da ritornare le info

DiGraph with 356467 nodes and 1667694 edges


In [33]:
degree_dict = dict(retweets_graph.degree(retweets_graph.nodes()))
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)

In [34]:
path[0]

'F:/Supsi/COVID-19-Tweets/large_files/rt_network/leiden/network_12/'

In [35]:
with open(path[0] + "degree.json", 'w') as f:
    json.dump(sorted_degree, f)

In [36]:
in_degree_dict = dict(retweets_graph.in_degree(retweets_graph.nodes()))
sorted_in_degree = sorted(in_degree_dict.items(), key=itemgetter(1), reverse=True)

In [37]:
with open(path[0] + "in_degree.json", 'w') as f:
    json.dump(sorted_in_degree, f)

In [38]:
out_degree_dict = dict(retweets_graph.out_degree(retweets_graph.nodes()))
sorted_out_degree = sorted(out_degree_dict.items(), key=itemgetter(1), reverse=True)

In [39]:
with open(path[0] + "out_degree.json", 'w') as f:
    json.dump(sorted_out_degree, f)

## Louvain

In [14]:
retweets_graph_und = nx.from_pandas_edgelist(df,'name','rt_name')
print(nx.info(retweets_graph_und)) 

Graph with 15740 nodes and 40686 edges


In [15]:
best_part = cm.best_partition(retweets_graph_und, random_state=42)

In [16]:
best_part

{'dad_liams': 0,
 'Just4TheCause': 0,
 'Patrici41859934': 0,
 'talialikeitis': 0,
 'JenFreedomFight': 0,
 'mombo3': 0,
 'KarinaRa1776': 0,
 'mamajay562': 0,
 'LoyolaTrue': 1,
 'HighWireTalk': 0,
 'uhavegot2bkidng': 0,
 'LifeLighted': 0,
 'BenSwann_': 2,
 'alumilynn': 0,
 'MehrtashOlson': 0,
 'valeeichner': 0,
 'FritzHousefritz': 0,
 'hellowolkenfeld': 0,
 'med1cinewoman': 0,
 'aparadiseearth7': 0,
 'cancervortex': 3,
 'drbloem': 3,
 'Kavvasakiman': 4,
 'WethepeopleF': 0,
 'JenniferMarguli': 0,
 'TallaTrialogue': 0,
 'DianeLeo': 0,
 '2ndfor1st': 0,
 'MargoGoida': 0,
 'twietsnest': 0,
 'Lex55747486': 0,
 'lorelfs1': 0,
 'Holly02375824': 0,
 'Louislane777': 0,
 'eroamy': 0,
 'newearthjourney': 0,
 'terriblewis1': 0,
 '118a53bf17b343a': 0,
 'Batoul____': 5,
 'reallygraceful': 5,
 'FreedomNY1': 0,
 'ConspiracyWATCH': 5,
 'TheCarissaB': 0,
 'ctmama4freedom': 0,
 'PJMoore1958': 0,
 'drsmit6': 0,
 'AlyssaKrumm': 0,
 'lightbearer0326': 0,
 'Prana4Love': 0,
 'niko319': 0,
 'patriot7842': 0,
 'ma

In [17]:
with open(path[0] + "louvain_communities.json", 'w') as f:
    json.dump(best_part, f)

In [18]:
path

['C:/Users/gianl/Desktop/Gi/Supsi/BachelorProject/network/network_2_6_0/']

# Leiden

In [40]:
tuples = [tuple(x) for x in df.values]
retweets_graph = ig.Graph.TupleList(tuples, directed = True)
n_vertices = retweets_graph.vcount()
print("Number of vertices:", n_vertices)
print("Number of edges:", retweets_graph.ecount())
print("Density of the graph:", 2*retweets_graph.ecount()/(retweets_graph.vcount()*(retweets_graph.vcount()-1)))
total_degree=0
total_indegree = 0
total_outdegree = 0
for n in range(n_vertices):
    total_degree += retweets_graph.degree(n)
    total_indegree += retweets_graph.indegree(n)
    total_outdegree += retweets_graph.outdegree(n)
print("Average indegree:", total_indegree/n_vertices)
print("Average outdegree:", total_outdegree/n_vertices)
partition = la.find_partition(retweets_graph, la.ModularityVertexPartition)
# retweets_graph.vs['name']
# partition.membership
zip_iterator = zip(retweets_graph.vs()["name"], partition.membership)
best_part = dict(zip_iterator)
best_part

Number of vertices: 356467
Number of edges: 2191443
Density of the graph: 3.4492345742748485e-05
Average indegree: 6.1476742587672915
Average outdegree: 6.1476742587672915


{'Terri55364130': 1,
 'waterlyll': 3,
 'wildwolf1313': 3,
 'BuyDirectUSA': 3,
 'Barbara_A_1977': 9,
 'g_easton819': 9,
 'dad_liams': 5,
 'Just4TheCause': 5,
 'formarrowdbs': 4,
 'EpochTimes': 9,
 '_YvonneBurton': 4,
 'talialikeitis': 5,
 'varks3': 1,
 'davdsoul': 213,
 'Patrici41859934': 5,
 'VeteransLawCtr': 1,
 'VeteransHealth': 11,
 'JenFreedomFight': 5,
 'mombo3': 5,
 'mdufay': 0,
 'AnnCoulter': 4,
 'PenelopeZahn': 26,
 'ThomasKlineMD': 26,
 'KarinaRa1776': 5,
 'mamajay562': 5,
 'annieone3': 0,
 'LoyolaTrue': 5,
 'HighWireTalk': 5,
 'RobertMEdwards3': 3,
 'ShellyKey': 3,
 'BrelynShockley': 2,
 '7avanna': 2,
 'ColleenEldrach1': 10,
 'queen_khalifia': 5,
 'med1cinewoman': 5,
 'uhavegot2bkidng': 5,
 'blaze0497': 3,
 'PatrioticJesse': 1,
 'BreitbartNews': 1,
 'VeBo1991': 3,
 'RedHoss44': 3,
 'SurvivorL': 5,
 'littlehoss16j': 3,
 'Wingman2606': 4,
 'sparklespp5': 26,
 'LifeLighted': 5,
 'BenSwann_': 5,
 'alumilynn': 5,
 'MehrtashOlson': 5,
 'old_atheist': 26,
 'DanLairdMD': 26,
 'IsIlli

In [41]:
with open(path[0] + "leiden_communities.json", 'w') as f:
    json.dump(best_part, f)